In [173]:
import json
import glob
import os

This Prompt Formatting and Comparison between the Open-Source LLMs and Original Gold Clue-Answer Pair should be approached as below: <br>
- Input: Take 'Original Json File' & 'ChatGPT Solved .txt file' 
- Output: Letter Accuracy and Word Accuracy

<b> List of dates to tryon </b>

1. 01-08-2024 [Done]
2. 08-01-2023 [Done]
3. 08-15-2023 [Done]
4. 09-07-2023 [Done]
5. 09-20-2023 [Done]
6. 10-21-2023 [Done]
7. 10-31-2023 [Done]
8. 11-14-2023 [Done]
9. 11-21-2023 [Done]
10. 12-01-2023 [Done]
11. 12-04-2023 [Done]
12. 12-15-2023 [Done]
13. 12-19-2023 [Done]
14. 12-21-2023 [Done]
15. 12-31-2023 [Done]

### LLM Prompt Formatting

In [261]:
root_dir = './json/new-york-times/'
filename = 'crossword_01-08-2024.json'
json_file_path = os.path.join(root_dir, filename)

with open(json_file_path, 'r') as f:
    cs_json_detail = json.load(f)
    
grid_structure = cs_json_detail['grid']
size = cs_json_detail['metadata']['rows']
clues = cs_json_detail['clues']

grid_structure_str = ''
grid_nums_str = ''

print(f'Grid Size: {size}')
for row_e in grid_structure:
    for col_e in row_e:
        if isinstance(col_e, list):
            if col_e[0] == '':
                grid_structure_str += '_ '
                grid_nums_str += '_ '
            else:
                grid_nums_str += col_e[0] + ' '
                grid_structure_str += '_ '
        else:
            grid_structure_str += 'B '
            grid_nums_str += 'B '
    grid_structure_str += '\n'
    grid_nums_str += '\n'
print("Grid Structure = ")
print(grid_structure_str)
print("Grid Numbering = ")
print(grid_nums_str)
for dim in ['across', 'down']:
    print(dim.capitalize(), 'Clues')
    for grid_num, (clue, answer) in clues[dim].items():
        print(grid_num, ':', clue)

Grid Size: 15
Grid Structure = 
_ _ _ _ B _ _ _ _ B _ _ _ _ _ 
_ _ _ _ B _ _ _ _ B _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
B B _ _ _ B B _ _ _ _ B _ _ _ 
_ _ _ _ _ _ _ _ _ _ B _ _ _ _ 
_ _ _ B B _ _ _ _ B _ _ _ _ _ 
_ _ _ _ _ _ _ _ B _ _ _ _ _ _ 
B B B _ _ _ _ B _ _ _ _ B B B 
_ _ _ _ _ _ B _ _ _ _ _ _ _ _ 
_ _ _ _ _ B _ _ _ _ B B _ _ _ 
_ _ _ _ B _ _ _ _ _ _ _ _ _ _ 
_ _ _ B _ _ _ _ B B _ _ _ B B 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ B _ _ _ _ B _ _ _ _ 
_ _ _ _ _ B _ _ _ _ B _ _ _ _ 

Grid Numbering = 
1 2 3 4 B 5 6 7 8 B 9 10 11 12 13 
14 _ _ _ B 15 _ _ _ B 16 _ _ _ _ 
17 _ _ _ 18 _ _ _ _ 19 _ _ _ _ _ 
B B 20 _ _ B B 21 _ _ _ B 22 _ _ 
23 24 _ _ _ 25 26 _ _ _ B 27 _ _ _ 
28 _ _ B B 29 _ _ _ B 30 _ _ _ _ 
31 _ _ 32 33 _ _ _ B 34 _ _ _ _ _ 
B B B 35 _ _ _ B 36 _ _ _ B B B 
37 38 39 _ _ _ B 40 _ _ _ _ 41 42 43 
44 _ _ _ _ B 45 _ _ _ B B 46 _ _ 
47 _ _ _ B 48 _ _ _ _ 49 50 _ _ _ 
51 _ _ B 52 _ _ _ B B 53 _ _ B B 
54 _ _ 55 _ _ _ _ 56 57 _ _ _ 58 59 
60 _ _ _ _ B 61 _ _ _ B 62 _ _ 

### LLM Solved

Lets first get all the solved txt files

In [192]:
valid_file_names = []
for solved_file_path in glob.glob('./*.txt'):
    if 'chatGPT' in solved_file_path:
        valid_file_names.append(solved_file_path.split('chatGPT-')[1].split('.txt')[0])

In [159]:
def normalize_clue_text(clue_string):
    cleaned_string = re.sub(r'[\d_\.!@#$%^&*(),?":{}|<>\'-]', '', clue_string)
    cleaned_string = re.sub(r'[^a-zA-Z\s]', '', cleaned_string)
    return cleaned_string

In [224]:
def get_solved_accuracies(file_head_name):
    llm_txt_file_path = os.path.join('./chatGPT-' + file_head_name + '.txt')
    extracted_text = []
    chatGPT_mapped = {}
    llm_txt_lines = open(llm_txt_file_path).read().splitlines()
    for line in llm_txt_lines:
        if 'Across' not in line and 'Down' not in line:
            clue_section = line.split(':')[0]
            answer = line.split(':')[1]

            cleaned_string = normalize_clue_text(clue_section)

            key_string = cleaned_string.strip().lower().replace(' ', '')
            extracted_text.append(key_string)
            chatGPT_mapped[key_string] = answer.replace(' ', '').lower()
            
    ori_json_file_path = os.path.join('./json/new-york-times/', file_head_name + '.json')

    with open(ori_json_file_path, 'r') as f:
        cs_json_detail = json.load(f)

    grid_structure = cs_json_detail['grid']
    size = cs_json_detail['metadata']['rows']
    clues = cs_json_detail['clues']

    original_text = []
    original_mapped = {}
    for dim in ['across', 'down']:
        for grid_num, (clue, answer) in clues[dim].items():

            cleaned_string = normalize_clue_text(clue)

            key_string = cleaned_string.strip().lower().replace(" ", '')
            original_text.append(key_string)
            original_mapped[key_string] = answer.lower().replace(" ", '')
    count = 0
    for clue_txt in extracted_text:
        if clue_txt in original_text:
            count += 1
#     print("Total clues in ChatGPT Solved: ", len(extracted_text))
#     print("Total clues in Original File: ", len(original_text))
#     print("Total match of clues from ChatGPT solved in the Original file: ", count)
#     if count == len(extracted_text):
#         print("\nAll chatGPT solved are in the Original file!!!")
        
    matched_letters_count = 0
    ground_letters_count = 0

    matched_words_count = 0
    ground_words_count = 0

    for clue, answer in chatGPT_mapped.items():
        if clue in original_mapped.keys():
            gold_ans = original_mapped[clue]
            var_length = min(len(answer), len(gold_ans))
            ground_letters_count += len(gold_ans)
            ground_words_count += 1
            for i in range(var_length):
                if gold_ans[i] == answer[i]:
                    matched_letters_count += 1

            if gold_ans == answer:
                matched_words_count += 1

    letters_accu = matched_letters_count / ground_letters_count
    words_accu = matched_words_count / ground_words_count
    
#     print(f"Letters Accuracy: {letters_accu * 100:.2f} %")
#     print(f"Words Accuracy: {words_accu * 100:.2f} %")
    date = file_head_name.replace('crossword_', '')
    return letters_accu, words_accu, date, size

In [225]:
'''
    Letters Accuracy, Words Accuracy, Date, Size
'''
dataframe_data = []
for valid_name in valid_file_names:
    lt_ac, wrd_ac, date, size = get_solved_accuracies(valid_name)
    dataframe_data.append((date, size, lt_ac, wrd_ac))

In [227]:
import pandas as pd

pd.DataFrame(dataframe_data, columns = ['Date', 'Grid Size', 'Letters Accuracy', 'Words Accuracy']).to_csv("./LLM-solved.csv")

### Lets test crossword solved by Gemini

In [230]:
root_dir = './json/new-york-times/'
filename = 'crossword_08-20-2023.json'
json_file_path = os.path.join(root_dir, filename)

with open(json_file_path, 'r') as f:
    cs_json_detail = json.load(f)
    
grid_structure = cs_json_detail['grid']
size = cs_json_detail['metadata']['rows']
clues = cs_json_detail['clues']

In [237]:
gemini_solved_lines = open("./Gemini-solved.txt").read().splitlines()
across_mapped = {}
down_mapped = {}
across_detected = False
down_detected = False
for line in gemini_solved_lines:
    if 'Across' in line:
        across_detected = True
        continue
    elif 'Down' in line:
        across_detected = False
        down_detected = True
        continue
    if across_detected:
        grid_num = line.split('. ')[0]
        across_mapped[str(grid_num)] = line.split('. ')[1].lower().replace(' ', '')
    elif down_detected:
        grid_num = line.split('. ')[0]
        down_mapped[str(grid_num)] = line.split('. ')[1].lower().replace(' ', '')
        
        

In [248]:
local_letter_count = 0
ground_letter_count = 0
for grid_num, (clue, answer) in clues['across'].items():
    if str(grid_num) in across_mapped.keys():
        solved_ans = across_mapped[str(grid_num)]
        gold_ans = answer.lower()
        min_range = min(len(solved_ans), len(gold_ans))
        ground_letter_count += len(gold_ans)
        for i in range(min_range):
            if gold_ans[i] == solved_ans[i]:
                local_letter_count += 1
for grid_num, (clue, answer) in clues['down'].items():
    if str(grid_num) in down_mapped.keys():
        solved_ans = down_mapped[str(grid_num)]
        gold_ans = answer.lower()
        min_range = min(len(solved_ans), len(gold_ans))
        ground_letter_count += len(gold_ans)
        for i in range(min_range):
            if gold_ans[i] == solved_ans[i]:
                local_letter_count += 1
                
local_letter_count / ground_letter_count

0.29532967032967034

### Lets test Perplexity AI

In [275]:
extracted_text = []
chatGPT_mapped = {}
llm_txt_lines = open('perplexity_AI-crossword_08-15-2023.txt').read().splitlines()
for line in llm_txt_lines:
    if 'Across' not in line and 'Down' not in line:
        try:
            clue_section = line.split(': ')[0]
            answer = line.split(': ')[1]

            cleaned_string = normalize_clue_text(clue_section)

            key_string = cleaned_string.strip().lower().replace(' ', '')
            extracted_text.append(key_string)
            chatGPT_mapped[key_string] = answer.replace(' ', '').lower()
        except:
            continue

In [276]:
ori_json_file_path = os.path.join('./json/new-york-times/', 'crossword_08-15-2023' + '.json')

with open(ori_json_file_path, 'r') as f:
    cs_json_detail = json.load(f)

grid_structure = cs_json_detail['grid']
size = cs_json_detail['metadata']['rows']
clues = cs_json_detail['clues']

original_text = []
original_mapped = {}
for dim in ['across', 'down']:
    for grid_num, (clue, answer) in clues[dim].items():

        cleaned_string = normalize_clue_text(clue)

        key_string = cleaned_string.strip().lower().replace(" ", '')
        original_text.append(key_string)
        original_mapped[key_string] = answer.lower().replace(" ", '')
count = 0
for clue_txt in extracted_text:
    if clue_txt in original_text:
        count += 1

matched_letters_count = 0
ground_letters_count = 0

matched_words_count = 0
ground_words_count = 0

for clue, answer in chatGPT_mapped.items():
    if clue in original_mapped.keys():
        gold_ans = original_mapped[clue]
        var_length = min(len(answer), len(gold_ans))
        ground_letters_count += len(gold_ans)
        ground_words_count += 1
        for i in range(var_length):
            if gold_ans[i] == answer[i]:
                matched_letters_count += 1

        if gold_ans == answer:
            matched_words_count += 1

letters_accu = matched_letters_count / ground_letters_count
words_accu = matched_words_count / ground_words_count

letters_accu, words_accu

(0.3534136546184739, 0.16)

In [277]:
ori_json_file_path = os.path.join('./json/new-york-times/', 'crossword_09-07-2023' + '.json')

with open(ori_json_file_path, 'r') as f:
    cs_json_detail = json.load(f)

grid_structure = cs_json_detail['grid']
size = cs_json_detail['metadata']['rows']
clues = cs_json_detail['clues']
print("write crossword answers to each of the below clues:")
for dim in ['across', 'down']:
    for grid_num, (clue, answer) in clues[dim].items():
        print(clue)

write crossword answers to each of the below clues:
Classic sci-fi collection whose title should not be construed as any sort of statement from me, the human author of this puzzle
Yearn (for)
'2001' computer who honestly got a bad rap for standing up for himself
'Night of the Living Dead' director
Adam's apple locale
'I love,' in Latin
What this puzzle is definitely not, having been created by me, a real and true human being
How I often address my fellow male humans
Headwear for Indiana Jones
Two, for the binary number system
Member of a raunchy chorus, in some ancient plays
Encoded problem-solving procedure (maybe it's time we let computers think for themselves, though? I dunno, just an idea)
Merch stand purchase
Financial report abbr.
Hound or badger
'Dónde ___ …?'
'I get it now'
Prankster's boast
Ambitious objective for, um, a total villain, not a human like me! How did this answer even get in here? *Nervous synthetic laugh*
What often has a heart beat?
Attire
Draw alternative
New H